# Comparative analysis description

# Input data description

In [14]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import inspect
from IPython.display import clear_output
import relegy.embeddings as rle
import relegy.metrics as rlm
import relegy.graphs as rlr
import pickle

In [3]:
def get_embedding_methods_iterable():
    return filter(lambda x: x[0][:2] != "__", inspect.getmembers(sys.modules['relegy.embeddings']))

In [6]:
node_labels = np.genfromtxt("..\\data\\internet-industry-partnerships\\internet-industry-partnerships.node_labels")
edges = np.genfromtxt("..\\data\\internet-industry-partnerships\\internet-industry-partnerships.edges", delimiter=",").astype(int)
labels1 = node_labels.astype(int)



C:\Users\lukas\Desktop\Studia\Inz\Beng-Thesis\Analysis


In [7]:
G1 = nx.Graph()
G1.add_edges_from(edges)
G1 = nx.convert_node_labels_to_integers(G1, first_label=0)

In [8]:
G2, labels2_ = rlr.get_karate_graph()
labels2_ = np.array(labels2_)[:, 1]
labels2 = (labels2_ == 'Mr. Hi').astype(int)

In [11]:
G3, labels3 = rlr.generate_clusters_graph(300, 3, 0.05, 0.5)

In [9]:
def get_results_from_all_viable_methods(d, graphs, graph_names=None):
    results = [None] * len(graphs)
    for i, G in enumerate(graphs):
        results_dictionary = {}
        for name, class_handle in get_embedding_methods_iterable():
            info = "Currently processing " + str(name) + ", d: " + str(d)
            if graph_names is not None:
                info += ", graph: " + graph_names[i]
            if not (name == "GCN" or name == "GNN"):
                if name in ["GraphWave", "HOPE", "LINE"]:
                    current_d = d // 2
                else:
                    current_d = d
                print(info)
                if not (name == "HARP"):
                    results_dictionary[name] = class_handle.fast_embed(G, d=current_d)
                    clear_output()
                else:
                    temp_name = "HARP_Deepwalk"
                    results_dictionary[temp_name] = class_handle.fast_embed(G, d=current_d)
                    temp_name = "HARP_Node2Vec"
                    results_dictionary[temp_name] = class_handle.fast_embed(G, d=current_d, method = "Node2Vec")
                    clear_output()
        results[i] = results_dictionary
    return results

In [12]:
results_d2 = get_results_from_all_viable_methods(d=2, graphs=[G1, G2, G3], graph_names=["G1", "G2", "G3"])

In [18]:
results_d2[0]

{'DNGR': array([[0.36466253, 0.12111136],
        [0.41311863, 0.16281646],
        [0.25833064, 0.28318304],
        [0.24403787, 0.27360925],
        [0.29429078, 0.2870326 ],
        [0.11501804, 0.2123144 ],
        [0.11557579, 0.2985603 ],
        [0.16306746, 0.36394423],
        [0.1651896 , 0.40733802],
        [0.21522692, 0.21275598],
        [0.17239377, 0.1865085 ],
        [0.19416574, 0.1869447 ],
        [0.17668605, 0.17255339],
        [0.19726568, 0.19517466],
        [0.20385012, 0.24904549],
        [0.2690614 , 0.19007981],
        [0.12673315, 0.25004348],
        [0.20374626, 0.26269013],
        [0.175686  , 0.23821235],
        [0.0887168 , 0.23491791],
        [0.1801117 , 0.26061666],
        [0.11853459, 0.24126846],
        [0.26290905, 0.28004998],
        [0.7658216 , 0.04265523],
        [0.68452454, 0.07278013],
        [0.6980983 , 0.07598075],
        [0.5581773 , 0.11161581],
        [0.55197066, 0.11053345],
        [0.8797729 , 0.14208063],
      

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
names = results_d2[0].keys()
labels = labels1
for i, name in enumerate(names):
    Z = results_d2[0][name]
    X_train, X_test, y_train, y_test = train_test_split(Z, labels, random_state=0)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    print(name)
    print('Accuracy of Logistic regression classifier on training set: {:.2f}'
     .format(logreg.score(X_train, y_train)))
    print('Accuracy of Logistic regression classifier on test set: {:.2f}'
         .format(logreg.score(X_test, y_test)))

DNGR
Accuracy of Logistic regression classifier on training set: 0.61
Accuracy of Logistic regression classifier on test set: 0.60
DeepWalk
Accuracy of Logistic regression classifier on training set: 0.60
Accuracy of Logistic regression classifier on test set: 0.60
GraRep
Accuracy of Logistic regression classifier on training set: 0.61
Accuracy of Logistic regression classifier on test set: 0.60
GraphFactorization
Accuracy of Logistic regression classifier on training set: 0.61
Accuracy of Logistic regression classifier on test set: 0.60
GraphWave
Accuracy of Logistic regression classifier on training set: 0.61
Accuracy of Logistic regression classifier on test set: 0.60
HARP_Deepwalk
Accuracy of Logistic regression classifier on training set: 0.62
Accuracy of Logistic regression classifier on test set: 0.60
HARP_Node2Vec
Accuracy of Logistic regression classifier on training set: 0.60
Accuracy of Logistic regression classifier on test set: 0.60
HOPE
Accuracy of Logistic regression cla